In [1]:
import os
import pandas as pd
import arcpy
from arcpy import env
from arcpy.management import *
from arcpy.sa import *
from arcpy.da import *
from arcpy.conversion import *
from arcpy.analysis import *

In [2]:
# tool function
# 创建渔网
def create_net(output_file,extent_file,cell_size):
    """参数说明：
        output_file: 输出文件
        extent_file: 输入范围文件
        cell_size: 生成的cell大小
    """
    proj = arcpy.Describe(extent_file).spatialReference
    extent_x_min = arcpy.Describe(extent_file).extent.XMin
    extent_y_min = arcpy.Describe(extent_file).extent.YMin
    extent_x_max = arcpy.Describe(extent_file).extent.XMax
    extent_y_max = arcpy.Describe(extent_file).extent.YMax
    with arcpy.EnvManager(outputCoordinateSystem = proj):
        arcpy.management.CreateFishnet(out_feature_class=output_file,origin_coord=f"{extent_x_min} {extent_y_min}",y_axis_coord=f"{extent_x_min} {extent_y_min+10}",cell_width=cell_size,cell_height=cell_size,number_rows=None,number_columns=None,corner_coord=f"{extent_x_max} {extent_y_max}",labels="NO_LABELS",template="#",geometry_type="POLYGON")
# 面转点
def polygon_point(in_feature, out_feature):
    """参数说明：
        in_feature: 输入面
        out_feature: 输出点
    """
    proj = arcpy.Describe(in_feature).spatialReference
    with arcpy.EnvManager(outputCoordinateSystem=proj):
        arcpy.management.FeatureToPoint(in_features=in_feature,out_feature_class=out_feature,point_location="INSIDE")

# 筛选点

def select_point(input_polygon,input_point,output_point):
    """参数说明：
        input_polygon: 选择面
        input_point: 待选择点
        output_point: 选择点
    """
    polygon_layer = "polygonLayer"
    point_layer = "pointLayer"
    arcpy.MakeFeatureLayer_management(input_polygon, polygon_layer)
    arcpy.MakeFeatureLayer_management(input_point, point_layer)
    arcpy.SelectLayerByLocation_management(in_layer=point_layer, overlap_type="INTERSECT", select_features=polygon_layer,search_distance=0,selection_type='NEW_SELECTION',invert_spatial_relationship="NOT_INVERT")
    # 导出选择的要素
    arcpy.CopyFeatures_management(point_layer, output_point)

# 添加字段
def add_field(input_table,field_name,field_type='TEXT'):
    """参数说明：
        input_table: 输入数据表
        field_name: 字段名
        field_type: 字段类型"""
    arcpy.AddField_management(input_table,field_name,field_type)

# 空间连接
def perform_spatial_join(target_layer_path, join_layer_path, output_layer_path, field_mapping_dict):
    """参数说明：
        target_layer_path: 目标图层路径
        join_layer_path: 连接图层路径
        output_layer_path: 输出图层路径
        field_mapping_dict: 字段映射字典
    """
    # 创建 FieldMappings 对象
    field_mappings = arcpy.FieldMappings()

    # 为每个字段对创建 FieldMap 并添加到 FieldMappings
    for target_field, join_field in field_mapping_dict.items():
        field_map = arcpy.FieldMap()
        
        # 添加目标字段和连接字段到FieldMap
        field_map.addInputField(target_layer_path, target_field)
        field_map.addInputField(join_layer_path, join_field)
        
        # 设置输出字段属性
        output_field = field_map.outputField
        output_field.name = target_field
        output_field.aliasName = target_field
        field_map.outputField = output_field

        # 将 FieldMap 添加到 FieldMappings
        field_mappings.addFieldMap(field_map)

    # 执行空间连接操作
    arcpy.analysis.SpatialJoin(target_layer_path, join_layer_path, output_layer_path,"JOIN_ONE_TO_ONE", "KEEP_ALL", field_mappings)


# 点采样
def sample_point(point_,raster_,out_name):
    """根据栅格采样点,输出为表格"""
    Sample(raster_,point_,out_name,"NEAREST", "OBJECTID", "CURRENT_SLICE", None, '', None, None, "ROW_WISE", "TABLE")
    return None

# 导出CSV
def export_csv(table_,out_path,out_name):
    """参数说明：
        table_: 输入数据表
        out_path: 输出路径
        out_name: 输出表名"""
    TableToTable(table_,out_path,out_name)
    return None


In [3]:
# 设置工作空间
env.workspace = r'D:\ArcGISProjects\workspace\shbyq\features_data_vector.gdb\features'


In [4]:
# 创建渔网
create_net('dunyun_grid','dy_tl_dh',300)

In [5]:
# 要素转点
polygon_point('dunyun_grid','dunyun_inside_point')

In [7]:
env.overwriteOutput=True

In [5]:
# 按位置选择
select_point('dy_tl_dh','dunyun_inside_point','select_point')

In [5]:
# 添加字段
input_features = 'select_point'
field_list = ['TL','YL','TS','TZ']
for one_field in field_list:
    # 添加字段
    add_field(input_table=input_features,field_name=one_field,field_type='TEXT')

In [7]:
# 空间连接赋予地类属性
# 定义目标要素图层和连接要素图层的路径
targetFeatures = "select_point"
joinFeatures = "dy_tl_dh"
outputFeatures = "join_point"
fields_mapping = {
    "TL": "修改代号",
    "YL": "修改代号",
    "TS": "修改代号",
    "TZ": "修改代号"
}
# 空间连接
perform_spatial_join(targetFeatures,joinFeatures,outputFeatures,fields_mapping)

# 采样环节

In [3]:
# 对特征因子采样
# result table name
sample_csv_name = 'dy_frequency_dz_dl.csv'
# 选择用于采样的特征因子数据库
stander_raster_gdb = r"D:\ArcGISProjects\workspace\shbyq\features_data_dy.gdb"
env.workspace = stander_raster_gdb # 切换工作空间用于采样
# 用于采样的特征因子列表
feature_list = arcpy.ListRasters()
len(feature_list)

37

In [4]:
# 选择用于采样的要素类
point_data = r"D:\ArcGISProjects\workspace\shbyq\features_data_vector.gdb\features\join_point_filter"

# 用于存放采样表的数据库路径
sample_gdb_path = r"F:\cache_data\frequency_filter\dy"

In [6]:
# 采样点数据名称
sample_name =os.path.basename(point_data)
filed_list = [_.name for _ in arcpy.ListFields(point_data)]
print(filed_list)
# 需要保留的字段
# elements_yes = ['OBJECTID', 'TL','YL','TS','TZ']
elements_yes = ['OBJECTID', 'TZ']
filter_list = [_ for _ in filed_list if _ in elements_yes]
print(filter_list)


['OBJECTID', 'Shape', 'Join_Count', 'TARGET_FID', 'TL', 'YL', 'TS', 'TZ']
['OBJECTID', 'TZ']


In [7]:
# 使用Delete_management函数删除数据库中的所有内容
try:
    arcpy.Delete_management(os.path.join(sample_gdb_path,'tabledata.gdb'))
except:
    pass
# 再创建一个新的数据库
arcpy.management.CreateFileGDB(sample_gdb_path, "tabledata", "CURRENT")

<Result 'F:\\cache_data\\frequency_filter\\dy\\tabledata.gdb'>

In [8]:
# 逐个采样并保存到csv文件
gdb_path = os.path.join(sample_gdb_path,"tabledata.gdb")
for one_feature in feature_list:
    print(one_feature)
    sample_point(point_data,one_feature,os.path.join(gdb_path,one_feature))

DEM
AnalyticalHillshading
Aspect
ChannelNetworkBaseLevel
ChannelNetworkDistance
ClosedDepressions
ConvergenceIndex
LSFactor
PlanCurvature
ProfileCurvature
RelativeSlopePosition
Slope
TopographicWetnessIndex
TotalCatchmentArea
ValleyDepth
Contrast
Correlation
Dissimilarity
Entropy
Homogeneity
Mean
ndvi
PCA_0
PCA_1
SecondMoment
Variance
PRE
SRA
TMP
VAP
WIND
BIO
LON
LAT
PH
DL
DZ


In [9]:
# 导出CSV表用于分析
env.workspace = gdb_path # 切换工作空间用于导出csv文件

In [10]:
# 读取数据表并保存到csv文件
result_df = pd.DataFrame(arcpy.da.FeatureClassToNumPyArray(point_data,filter_list))
result_df.rename(columns={"OBJECTID":sample_name},inplace=True)

In [11]:
#  读取每个表的最后一个字段的数据,存储每个表的最后一个字段的数据
for table in feature_list:
    # 将表转换为pandas数据帧
    df = pd.DataFrame(arcpy.da.TableToNumPyArray(table, "*"))  # 确保数据表中无空值
    # 提取最后一个字段的数据
    merged_df = df[[sample_name, df.columns[-1]]]
    # 合并
    result_df = pd.merge(result_df, merged_df, on=[sample_name])
# 保存到csv文件
result_df.rename(columns=dict(zip(result_df.columns[-len(feature_list):], feature_list)),inplace=True)
result_df.drop(result_df.columns[0],axis=1,inplace=True)
# result_df.rename(columns={"PH":"PH_T"},inplace=True)
result_df.to_csv(os.path.join(sample_gdb_path,sample_csv_name),index=False,encoding='utf-8')

In [12]:
df = pd.read_csv(os.path.join(sample_gdb_path,sample_csv_name))
df

,TZ,DEM,AnalyticalHillshading,Aspect,ChannelNetworkBaseLevel,ChannelNetworkDistance,ClosedDepressions,ConvergenceIndex,LSFactor,PlanCurvature,...,SRA,TMP,VAP,WIND,BIO,LON,LAT,PH,DL,DZ
0,13d,961.5,0.706572,0.231111,827.27014,134.229860,NaN,16.303608,1.840937,0.024353,...,13154.333,15.766666,1.555833,1.658333,194.86105,443245.0,2861300.0,5.355657,3.0,15
1,106,888.4,0.426612,6.042653,870.42035,17.979675,NaN,2.823421,13.958138,0.009785,...,13154.333,15.766666,1.555833,1.658333,194.86105,443545.0,2861300.0,5.901145,3.0,48
2,13d,870.8,1.131889,1.264116,835.30994,35.490050,NaN,-2.361449,18.948938,-0.010550,...,13154.333,15.766666,1.555833,1.658333,194.86105,443845.0,2861300.0,5.594585,1.0,48
3,13d,1021.6,0.186124,5.645218,940.00836,81.591610,NaN,4.197873,13.658210,0.010529,...,13140.667,15.600000,1.540833,1.683333,195.54823,444445.0,2861300.0,5.476191,3.0,48
4,63f,841.3,0.541649,0.017862,800.18420,41.115784,NaN,0.775370,18.760067,0.002961,...,13150.667,15.750000,1.551667,1.675000,194.79027,442345.0,2861600.0,5.695483,3.0,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23193,14d,946.0,0.791960,3.165428,946.40485,0.000000,0.404846,-0.340999,3.453542,0.000774,...,12838.583,14.975000,1.472500,1.741667,189.16838,449845.0,2923400.0,6.063271,1.0,16
23194,14d,1000.0,1.232808,3.646036,971.13696,28.863037,NaN,2.365763,28.606610,0.017944,...,12832.167,14.908333,1.464167,1.750000,189.22100,450145.0,2923400.0,5.685568,3.0,5
23195,14d,1168.3,0.968874,1.988977,1096.30960,71.990480,NaN,26.080515,2.520325,0.023970,...,12810.917,14.716667,1.446667,1.758333,189.39268,448645.0,2923700.0,5.808790,NaN,18
23196,14d,1043.2,0.998859,3.105903,1019.76750,23.432434,NaN,2.689742,9.056221,0.004753,...,12838.583,14.975000,1.472500,1.741667,189.16838,449545.0,2923700.0,5.624920,3.0,16
